In [1]:
import requests

import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

import csv
import re
import urllib.parse
import io
import os.path
import os
import random
random.seed(853915)

In [2]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

In [3]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams) #, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

In [4]:
url = 'http://www.congreso.gob.pe/Docs/participacion/museo/congreso/files/mensajes/2001-2020/files/mensaje-2019-01-vizcarra.pdf'
file = '../week-1/mensaje-2019-01-vizcarra.pdf'
speech = readPDF(downloadIfNeeded(url, file, mode = 'rb'))

In [5]:
speech[:500]

'DISCURSO DEL PRESIDENTE CONSTITUCIONAL DE LA REPÚBLICA, \n\nINGENIERO MARTÍN ALBERTO VIZCARRA CORNEJO, \n\nANTE EL CONGRESO NACIONAL, EL 28 DE JULIO DE 20191 \n\n \n \nSeñor Presidente del Congreso de la República; \nSeñoras y señores congresistas; \nSeñoras y señores ministros de Estado; \nDistinguidas autoridades civiles y militares; \nMiembros del cuerpo diplomático acreditado en el Perú; \nCompatriotas, a lo largo y ancho del país: \n \nDe conformidad al artículo 118 de la Constitución Política del Estado,'

In [6]:
sentences1 = re.findall('[A-Z¿¡][^.]*\.',speech, re.DOTALL)

In [7]:
sentences = []
for i, sentence in enumerate(sentences1):
    new_sentence = re.sub('\n\n \n\n[0-9]+ \n\n\x0c', '', sentence)
    new_sentence = re.findall('[A-Z¿¡](?!.*\n \n).*\.', new_sentence, re.DOTALL)[0]
    new_sentence = re.sub('\n', '', new_sentence)
    if not 'https://' in new_sentence:
        sentences.append(new_sentence)

In [8]:
sentences[90:110]

['Las  vías  de  comunicación  generan  desarrollo,  facilitan  el traslado de los productos, fomentan el turismo; y además, son elemento integrador para  las  familias,  porque  permiten  acortar  distancias,  mejorar  la  salud  y  la educación.',
 'En  el  presente  año  estamos  pavimentando  más  de  mil  doscientos  kilómetros  de carreteras, con una inversión que supera los 3,400 millones de soles en doce obras, como parte de la Red Vial Nacional, que tiene el 80% de pavimento, de sus veintiséis mil  ochocientos  kilómetros  que  la  conforman.',
 'A  este  ritmo  la  Red  Vial  Nacional estará pavimentada al 100% en cinco años.',
 'Sin embargo, donde hay un enorme retraso es en la Red Vial Departamental, que solo tiene el 13% de pavimento de los veintisiete mil quinientos kilómetros que la conforman.',
 'Por ello, anuncio el Proyecto de Infraestructura de la Red Vial Departamental, que incluye  quince  mil  kilómetros  de  pavimento,  uniendo  cuarenta  y  dos  corredores logís

In [9]:
num_sample = 100

In [10]:
samples = []
max_number = len(sentences) - 1
while len(samples) < num_sample:
    number = random.randint(0, max_number)
    if not number in samples:
        samples.append(number)

In [11]:
sentences_sample = [sentences[i] for i in samples]

In [12]:
sentences_sample[:20]

['Este número es importante, pero insuficiente, dada la gran necesidad de titulación que existe en el Perú.',
 'Tenemos  mayor  celeridad  en  procesos  contenciosos  administrativos judiciales gracias a la eliminación de la emisión del dictamen fiscal.',
 'Hoy  Lima  2019  está  cambiando  positivamente  la  forma  en  que  se  realizan,  se disfrutan  y  se  organizan  eventos  deportivos  en  el  Perú.',
 'Por ello, la descentralización ha sido, es y seguirá siendo nuestro sello de gobierno.',
 'Durante el 2018 el PBI del sector agropecuario creció 7.',
 'Algunas  autoridades  del  más  alto  nivel  nos estaban robando el país.',
 'Sin embargo, la norma que la rige tiene cerca de treinta años de vigencia, y se hace necesario actualizarla para  que  se  adecúe  a  nuestra  realidad,  brindando  un  marco  legal  claro  a  los inversionistas, y dando tranquilidad y desarrollo para todos.',
 'Hoy  el  Perú  ya  cuenta  con  un  nuevo  y  moderno  Centro  de Operaciones  de  Emergencia 

In [13]:
len(sentences_sample)

100

In [14]:
sample_file = 'sample_sentences.csv'
with open(sample_file, 'w', newline='') as file:
    wr = csv.writer(file)
    wr.writerow(['N','sentence'])
    for i, sentence in enumerate(sentences_sample):
        wr.writerow([i + 1, sentence])

In [15]:
others_file = 'uncoded_sentences.csv'
with open(others_file, 'w', newline='') as file:
    wr = csv.writer(file)
    wr.writerow(['N','sentence'])
    i = 1
    for sentence in sentences:
        if not sentence in sentences_sample:
            wr.writerow([i, sentence])
            i += 1